In [23]:
import numpy as np
import pandas as pd
import calendar

In [69]:
#import data
#All data are as of 02/14/2020 after market close 

#Range of futures maturities
#futures price: March, May, July, October
T = [3, 5, 8]
current_month = 2

#LIBOR rate

#using linear interpolation for 5M and 8M libor
libor_term = [1, 2, 3, 6, 12]
libor_rate = [1.65825, 1.6625, 1.69175, 1.71, 1.79838] 
date = pd.to_datetime(['2020-02-14', '2020-05-01', '2020-07-01', '2020-10-01'])
date_diff = [(date[1]-date[0]).days/31, (date[2]-date[0]).days/31, (date[3]-date[0]).days/31]
rate = np.interp(date_diff, libor_term, libor_rate)

libor = pd.DataFrame({'Term': T, 'Rate':rate})

#futures price: March, May, July, October
future = pd.DataFrame(data = {'maturity':T, 'price':[14.55, 14.45, 14.48]})

#spot as of 02/14/2020
s0 = 14.63


In [75]:
#Get the fair values of the future contracts 
fair_value = {}
n = len(future.loc[:,'maturity'])
for m in range(n):
    fv = s0*np.exp((libor.iloc[m,0]/12)*(libor.iloc[m,1]/100))
    diff = fv - future.iloc[m,1]
    perc_diff = diff/future.iloc[m,1]
    exp_month = calendar.month_abbr[T[m] + current_month]
    fair_value[str(T[m])+'M'] = [exp_month, fv, diff, perc_diff]
fairValue = pd.DataFrame(fair_value, index = ['exp_month', 'fair_value', 'diff(fair-future)', 'percentage_diff']).T
print(fairValue)

   exp_month fair_value diff(fair-future) percentage_diff
3M       May    14.6915          0.141452      0.00972181
5M       Jul     14.734          0.284033       0.0196562
8M       Oct    14.7998          0.319799       0.0220856


In [71]:
#Solve for cost of carry
def solver(S0, F, T):
    return np.log(F/S0)/T

In [79]:
#Solve for c-y and u-y
#u-y helps to to draw inferences about the relative cost of storing your commodity to the relative utility that the market assigns to holding the underlying asset 
cost_of_carry = {}
for m in range(n):
    c = solver(s0, future.iloc[m,1], date_diff[m]/12)
    cost_of_carry[str(future.iloc[m,0])+'M']= [c, -(c-libor.iloc[m,1]/100)]
costCarry = pd.DataFrame(cost_of_carry, index = ['net_cost_of_carry', 'relative_utility']).T
print(costCarry)

    net_cost_of_carry  relative_utility
3M          -0.026490          0.043257
5M          -0.033372          0.050377
8M          -0.016669          0.033978
